#####  环境：阿里云 ubuntu  python27 tensorflowf1.4.0

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.13.1'

In [3]:
![789](../TfInActionImg/1313.PNG) 图片插入示例

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


## 5.1 MNIST 数据处理

#### 虽然MNIST数据集只提供了训练和测试数据，但是为了验证模型训练效果，一般会从训练数据中划分出一部分数据作为验证数据。

#### Tensorflow提供了一个类处理该数据集，它会自动下载并转化MNIST数据的格式，下面给出它的样例程序

In [5]:
# TensorFlow的MNIST类样例

from tensorflow.examples.tutorials.mnist import input_data

#载入数据，如果指定路径无该数据集，则会自动下载
mnist = input_data.read_data_sets("../data/mnist/",one_hot=True)

# 打印训练数据大小
print ("Training data size:",mnist.train.num_examples)

# 打印验证数据大小
print ("Validating data size:",mnist.validation.num_examples)

# 打印测试数据大小
print ("Testing data size:",mnist.test.num_examples)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Training data size: 55000
Validating data size: 5000
Testing data size: 10000


In [7]:
# 打印训练数据示例
print ("Training data example:",mnist.train.images[0])

Training data example: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        

In [8]:
# 打印训练数据标签
print ("Training data label:",mnist.train.labels[0])

Training data label: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


![1356](../TfInActionImg/1356.PNG)

In [10]:
batch_size = 100
xs,ys = mnist.train.next_batch(batch_size)
print ("X shape:",xs.shape)
print ("Y shape:",ys.shape)

X shape: (100, 784)
Y shape: (100, 10)


In [11]:
print ("Training data size:",mnist.train.images[55].shape)

Training data size: (784,)


## 5.2 神经网络模型训练以及不同模型结果对比

### 5.2.1 TensorFlow训练神经网络

In [ ]:
# 运用第四章的方法，给出一个完整的神经网络程序
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNIST数据集相关的常数
INPUT_NODE = 784                    #输入层节点数，对于该数据集相当于图片像素28*28
OUTPUT_NODE = 10                    #输出层节点数，该数据集输出分为0~9类

#配置神经网络参数
LAYER1_NODE = 500                   #隐藏层节点数，用只有一个隐藏层的网络结构作为样例
BATCH_SIZE  = 100                   #数字越小，越接近随机梯度下降；越大越接近梯度下降
LEARNING_RATE_BASE = 0.8            #基础学习率
LEARNING_RATE_DECAY = 0.99          #学习率的衰减率
REGULARIZATION_RATE = 0.0001        #描述模型复杂程度的正则化项在损失函数中的系数
TRAINING_STEPS      = 30000         #训练轮数
MOVING_AVERAGE_DECAY = 0.99         #滑动平均衰减率

#定义一个使用relu激活函数的三层全连接神经网络
def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    #当没有使用滑动平均类的时候直接使用参数的当前取值
    if avg_class == None:
        # 计算隐藏层的前向传播结果
        layer1= tf.nn.relu(tf.matmul(input_tensor,weights1) + biases1)
        #计算输出层的前向传播结果。计算损失函数时候会一起计算softmax函数，所以这里无激活
        #函数。因为预测使用的是不同类别对应节点的输出值相对大小，有没有softmax层
        #对最后分类结果计算没有影响
        return tf.matmul(layer1,weights2) + biases2
    
    else:
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor,avg_class.average(weights1)) + \
            avg_class.average(biases1)
        )
        return tf.matmul(layer1,avg_class.average(weights2)) + \
            avg_class.average(biases2)
    
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name = 'x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name = 'y-input')
    
    #生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape = [LAYER1_NODE]))
    
    #生成输出层参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1,shape = [OUTPUT_NODE]))
    
    #计算如下参数下前向传播结果
    y = inference(x,None,weights1,biases1,weights2,biases2)
    
    #定义训练轮数的变量。这个变量不需要计算滑动平均值，所以指定变量为不可训练
    global_step = tf.Variable(0,trainable=False)
    
    #给定华东平均衰减率和训练轮数的变量，初始化华东平均类
    #？？因为给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
    # 在代表神经网络参数的变量上使用滑动平均
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 计算滑动平均之后的前向传播结果
    average_y = inference(x,variable_averages,weights1,biases1,weights2,biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularization
    # 设置指数衰减的学习率
    
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,global_step,
        mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY
    )
    
    # 批梯度下降优化
    train_step = tf.train.GradientDescentOptimizer(learning_rate)\
    .minimize(loss,global_step = global_step)
    
    #反向传播更新神经网络参数，还要更新每个参数滑动平均值
    # train_op = tf.group(train_step,variables_averages_op)#同下面两行等价
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op = tf.no_op(name = "train")
    
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.arg_max(y_,1))
    #将布尔数值转换为实数数值，然后计算平均值即为一组数值上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        #准备验证数据
        #通过验证数据大致判断停止的条件和评判训练效果
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        
        #准备测试数据
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        
        #迭代训练神经网络
        for i in range(TRAINING_STEPS):
            if i%1000 == 0:
                validate_acc = sess.run(accuracy,feed_dict = validate_feed)
                print("After %d training step(s), validation accuracy"
                     "using average model is %g"%(i,validate_acc))
            #产生这一轮使用的一个batch的数据，并运行训练过程
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})
        
        #在训练结束后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print("After %d training step(s), test accuracy using average"
                     "model is %g"%(TRAINING_STEPS,test_acc))
        
#主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("../data/mnist/",one_hot=True)
    train(mnist)
    
#Tensorflow提供一个主程序入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracyusing average model is 0.095
After 1000 training step(s), validation accuracyusing average model is 0.976
After 2000 training step(s), validation accuracyusing average model is 0.9816
After 3000 training step(s), validation accuracyusing average model is 0.9822
After 4000 training step(s), validation accuracyusing average model is 0.9832
After 5000 training step(s), validation accuracyusing average model is 0.9836
After 6000 training step(s), validation accuracyusing average model is 0.9832
After 7000 training step(s), validation accuracyusing average model is 0.9838
After 8000 training step(s), validation accuracyusing average model is 0.9836
After 9000 training step(s), validation accuracyusing average model is 0.9834
After 10000 trainin

### 5.2.2 使用验证数据集判读模型效果

#### 为了估计模型在未知数据的效果，需要保证测试数据在训练过程不可见

#### 除了使用验证数据集，还可以使用交叉验证

### 5.2.3 不同模型效果比较

![1620](../TfInActionImg/1620.PNG)
![1626](../TfInActionImg/1626.PNG)

#### 从下图看出，只优化交叉熵损失虽然在训练数据上比优化总损失模型更小，但是在测试数据上的正确率却不好。

![1630](../TfInActionImg/1630.PNG)

## 5.3 变量管理

In [14]:
import tensorflow as tf

#### 创建变量

In [16]:
# 下面两个定义时等价的
vX= tf.get_variable("vX",shape=[1],initializer=tf.constant_initializer(1.0))#只能执行一次
v = tf.Variable(tf.constant(1.0,shape=[1]),name="v")
print (vX)
print (v)

<tf.Variable 'vX:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'v:0' shape=(1,) dtype=float32_ref>


In [17]:
tf.__version__

'1.13.1'

![0746](../TfInActionImg/0746.PNG)

#### tf.get_variable首先会创建一个名字为vX的参数，如果创建失败（比如有同名）就报错


In [18]:
# 通过tf.variable_scope函数控制tf.get_variable函数获取已经创建的变量

#在名字为foo的命名空间内创建名字为v的变量
with tf.variable_scope("foo"):
    v = tf.get_variable("v",[1],initializer = tf.constant_initializer(1.0))

In [ ]:
# 因为在命名空间已经存在名字为v的变量所以下面代码会报错
with tf.variable_scope("foo"):
    v = tf.get_variable("v",[1])

In [21]:
# 获取已经声明的变量
with tf.variable_scope("foo",reuse=True):
    v1 = tf.get_variable("v",[1])
    print (v ==v1)

True


In [27]:
# !!bug code
# bar 中没有创建v，所以下面代码会报错
with tf.variable_scope("bar",reuse=True):
    v1 = tf.get_variable("v",[1])

#### resue取值：
![0816](../TfInActionImg/0816.PNG)

#### tf.variable_scope管理命名空间：
![0818](../TfInActionImg/0818.PNG)
![0822](../TfInActionImg/0822.PNG)

In [ ]:
v1 = tf.get_variable("v",[1],reuse=True)
print(v1.name)

## 5.4 TensorFlow模型持久化

### 5.4.1 持久化代码的实现

In [20]:
import tensorflow as tf

# 清除默认图的堆栈，并设置全局图为默认图 
tf.reset_default_graph()

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "v2")
result = v1+v2

init_op =  tf.global_variables_initializer()
#声明该类保存模型
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess,"./model/model.ckpt")

In [21]:
# 将上面生成的持久化模型加载

import tensorflow as tf

# 清除默认图的堆栈，并设置全局图为默认图 
tf.reset_default_graph()

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "v2")
result = v1+v2

#声明该类保存模型
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess,"./model/model.ckpt")
    print sess.run(result)

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
[3.]


In [22]:
print result

Tensor("add:0", shape=(1,), dtype=float32)


#### 直接加载已经持久化的图，可以省却重复定义图上的运算

In [23]:
import tensorflow as tf 
# 清除默认图的堆栈，并设置全局图为默认图 
tf.reset_default_graph()
saver = tf.train.import_meta_graph("./model/model.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess,"./model/model.ckpt")
    print sess.run(tf.get_default_graph().get_tensor_by_name("add:0"))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
[3.]


#### tf.train.Saver()不仅支持加载部分变量还支持变量重命名

In [24]:
import tensorflow as tf

# 清除默认图的堆栈，并设置全局图为默认图 
tf.reset_default_graph()

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "other-v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "other-v2")
result = v1+v2

#声明该类保存模型
saver = tf.train.Saver({"v1":v1,"v2":v2})

with tf.Session() as sess:
    saver.restore(sess,"./model/model.ckpt")
    print sess.run(result)

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
[3.]


#### 这样做，可以使我们很方便使用滑动平均值

In [25]:
# 保存滑动平均模型示例

import tensorflow as tf


v = tf.Variable(0, dtype=tf.float32, name="v")
for variables in tf.global_variables(): 
    print variables.name
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
for variables in tf.global_variables():
    print variables.name

saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    # 保存的时候会将v:0  v/ExponentialMovingAverage:0这两个变量都存下来。
    saver.save(sess, "Saved_model/model2.ckpt")
    print sess.run([v, ema.average(v)])

other-v1:0
other-v2:0
v:0
other-v1:0
other-v2:0
v:0
other-v1/ExponentialMovingAverage:0
other-v2/ExponentialMovingAverage:0
v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [26]:
# 加载

v = tf.Variable(0, dtype=tf.float32, name="v")

# 通过变量重命名将原来变量v的滑动平均值直接赋值给v。
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model2.ckpt")
    print sess.run(v)

INFO:tensorflow:Restoring parameters from Saved_model/model2.ckpt
0.099999905


#### variables_to_restore应用示例

In [27]:
import tensorflow as tf
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
print ema.variables_to_restore()

saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model2.ckpt")
    print sess.run(v)

{u'v_1/ExponentialMovingAverage': <tf.Variable 'v_1:0' shape=() dtype=float32_ref>, u'other-v1/ExponentialMovingAverage_1': <tf.Variable 'other-v1:0' shape=(1,) dtype=float32_ref>, u'v/ExponentialMovingAverage_1': <tf.Variable 'v:0' shape=() dtype=float32_ref>, u'other-v2/ExponentialMovingAverage': <tf.Variable 'other-v2/ExponentialMovingAverage:0' shape=(1,) dtype=float32_ref>, u'other-v1/ExponentialMovingAverage': <tf.Variable 'other-v1/ExponentialMovingAverage:0' shape=(1,) dtype=float32_ref>, u'v_2/ExponentialMovingAverage': <tf.Variable 'v_2:0' shape=() dtype=float32_ref>, u'other-v2/ExponentialMovingAverage_1': <tf.Variable 'other-v2:0' shape=(1,) dtype=float32_ref>, u'v/ExponentialMovingAverage': <tf.Variable 'v/ExponentialMovingAverage:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from Saved_model/model2.ckpt
0.099999905


#### tf.train.Saver会保存运行tf所需的所有信息，但有时并不需要所有信息。
#### 可以用convert_variables_to_constants将计算图中的变量和取值通过常数的方式保存

In [28]:
# 保存pb
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    with tf.gfile.GFile("Saved_model/combined_model.pb", "wb") as f:
           f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [29]:
# 加载pb
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename = "Saved_model/combined_model.pb"
   
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print sess.run(result)

[array([3.], dtype=float32)]


### 5.4.2 持久化数据存储

#### saver.save生成四个文件

MetaGraphDef

In [30]:
# 导出该文件的json模式
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2 = tf.Variable(tf.constant(2.0,shape=[1]),name="v2")
result1 = v1 + v2

saver = tf.train.Saver()
saver.export_meta_graph("./model/model.ckpt.meda.json",as_text = True)

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        is_ref: true
      }
      att

属性具体介绍（待看）

## 5.5 TensorFlow最佳实践样例程序

#### 5.2.1 程序的缺点;
- 前向传播变量传入太复杂，可扩展性差；
- 训练好的模型没有保存
- 训练测试没有分离

In [34]:
# mnist_inference定义前向传播过程以及神经网络参数
#定义神经网络结构相关的参数
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
#通过tf.get_variable函数来获取变量。
def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses', regularizer(weights))
    return weights

#定义神经网络的前向传播过程。
def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):

        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)


    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2

In [35]:
# mnist_train定义神经网络训练过程
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

# 定义神经网络结构相关的参数
BATCH_SIZE = 100 
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 5000
MOVING_AVERAGE_DECAY = 0.99 
MODEL_SAVE_PATH = "MNIST_model/"
MODEL_NAME = "mnist_model"

# 定义训练过程
def train(mnist):
    # 定义输入输出placeholder。
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)
    
    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

#主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("./data/mnist", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
After 1 training step(s), loss on training batch is 2.83118.
After 1001 training step(s), loss on training batch is 0.233557.
After 2001 training step(s), loss on training batch is 0.166953.
After 3001 training step(s), loss on training batch is 0.144474.
After 4001 training step(s), loss on training batch is 0.138774.


In [37]:
# mnist_eval 定义测试过程
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import mnist_train

# 加载的时间间隔。
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}

        y = mnist_inference.inference(x, None)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(EVAL_INTERVAL_SECS)
            
def main(argv=None):
    mnist = input_data.read_data_sets("../data/mnist/", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    main()

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from MNIST_model/mnist_model-4001
After 4001 training step(s), validation accuracy = 0.9818
INFO:tensorflow:Restoring parameters from MNIST_model/mnist_model-4001
After 4001 training step(s), validation accuracy = 0.9818
INFO:tensorflow:Restoring parameters from MNIST_model/mnist_model-4001
After 4001 training step(s), validation accuracy = 0.9818
INFO:tensorflow:Restoring parameters from MNIST_model/mnist_model-4001
After 4001 training step(s), validation accuracy = 0.9818


KeyboardInterrupt: 